In [15]:
import pandas as pd
import re

In [16]:
df_games = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_games_clean.parquet')
df_items = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_items_clean.parquet')
df_reviews = pd.read_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_reviews_clean.parquet')


df_games_copy = df_games.copy()
df_reviews_copy = df_reviews.copy()
df_sentimientos_copy = df_sentimientos.copy()


In [17]:
df_sentimientos_copy

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,"Posted November 5, 2011.",1250.0,True,2
1,76561197970982479,"Posted July 15, 2011.",22200.0,True,2
2,76561197970982479,"Posted April 21, 2011.",43110.0,True,2
3,js41637,"Posted June 24, 2014.",251610.0,True,2
4,js41637,"Posted September 8, 2013.",227300.0,True,2
...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70.0,True,2
59329,76561198312638244,Posted July 8.,362890.0,True,2
59330,LydiaMorley,Posted July 3.,273110.0,True,2
59331,LydiaMorley,Posted July 20.,730.0,True,2


In [18]:
#Assuming your DataFrames are named df1 and df2 (replace if necessary)
#Merge df1 and df2 on item_id using left join (keep all rows from df1)
df_merged = df_sentimientos_copy.merge(df_games_copy[['id', 'developer']], left_on='item_id', right_on='id', how='left')

#Handle potential missing values in the developer column (if item_id is not found in df2)
#df_merged['developer'] = df_merged['developer'].fillna('Developer Not Found')  # Or any other default value

#Print the resulting DataFrame
df_merged

ValueError: You are trying to merge on float64 and object columns for key 'item_id'. If you wish to proceed you should use pd.concat

In [ ]:

# Define a regex pattern to extract the year (four digits)
pattern = r'(\b\d{4}\b)'

# Extract the year from each cell in the 'posted' column using regex
df_merged['year_posted'] = df_merged['posted'].str.extract(pattern)

# Drop the original 'posted' column
df_merged.drop(columns=['posted'], inplace=True)



In [ ]:
nuevo_orden_columnas = [
    'user_id',
    'item_id',
    'recommend',
    'sentiment_analysis',
    'developer',
    'year_posted'
]

# Reordena las columnas del DataFrame con el nuevo orden
df_merged = df_merged[nuevo_orden_columnas]

df_merged

In [ ]:
# Drop rows with NaN values in 'year_posted' column
df_merged.dropna(subset=['year_posted'], inplace=True)

# Convert 'year_posted' column to integer data type
df_merged['year_posted'] = df_merged['year_posted'].astype(int)



In [ ]:
df_merged.to_parquet(r'C:\Users\beatr\OneDrive\Escritorio\henry\Proyecto Steam\Consulta_Data\df_merged.parquet')

In [ ]:
def best_developer_year(df, año):
    """
    Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado.
    Se tienen en cuenta recommend = True + sentiment_analysis con mayor puntuación.

    Args:
        df (pd.DataFrame): DataFrame que contiene los datos.
        año (int): Año para el que se desea obtener el top 3 de desarrolladores.

    Returns:
        dict: Diccionario ordenado con los top 3 desarrolladores para el año dado, en formato {1er puesto: primer juego, 2do puesto: segundo juego, 3er puesto: tercer juego}.
    """
    # Filtrar el DataFrame para el año dado
    df_año = df[df['year_posted'] == año]

    # Filtrar por recomendaciones verdaderas y sentimiento de análisis más alto
    df_filtrado = df_año[df_año['recommend'] & (df_año['sentiment_analysis'] == df_año['sentiment_analysis'].max())]

    # Agrupar por desarrollador y contar las recomendaciones
    top_developers = df_filtrado.groupby('developer')['recommend'].sum()

    # Ordenar los desarrolladores por número de recomendaciones en orden descendente
    top_developers = top_developers.sort_values(ascending=False)

    # Tomar los top 3 desarrolladores
    top_3_developers = top_developers.head(3)

    # Crear el diccionario ordenado con los top 3 desarrolladores
    top_developers_dict = {}
    for i, (developer, recomendaciones) in enumerate(top_3_developers.items(), 1):
        puesto = f"{i} puesto"
        top_developers_dict[puesto] = developer

    return top_developers_dict

In [ ]:
# Llamar a la función con el año deseado
año = 2012
top_3_desarrolladores = best_developer_year(df_merged, año)

# Imprimir el top 3 de desarrolladores para el año dado
print(f"Top 3 de desarrolladores para el año {año}: {top_3_desarrolladores}")


Top 3 de desarrolladores para el año 2012: {'1 puesto': 'Valve', '2 puesto': 'Re-Logic', '3 puesto': 'Facepunch Studios'}
